In [1]:
from __future__ import print_function

import argparse
import os
from tqdm import tqdm
import time
import random
import numpy as np
import wandb

import torch
import torch.backends.cudnn as cudnn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100

from models import prompters
from utils import accuracy, AverageMeter, ProgressMeter, save_checkpoint, cosine_lr, refine_classname

In [15]:
class Args:
    print_freq = 10
    save_freq = 509
    batch_size = 128
    num_workers = 16
    epochs = 1000
    optim='sgd'
    learning_rate= 40
    weight_decay = 0
    warmup = 1000
    momentum = 0.9
    patience = 1000
    model = None
    method = 'padding'
    prompt_size = 30
    root = './data'
    dataset = 'cifar100'
    image_size = 224
    seed = 0
    model_dir = './save/models'
    image_dir = './save/images'
    filename = None
    trial = 1
    resume = None
    evaluate = False
    gpu = None
    use_wandb = False
    
    

args=Args()

In [17]:
device = "cuda:0"

In [18]:
model = models.__dict__['resnet50'](pretrained=True).to(device)

In [19]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [20]:
prompter = prompters.__dict__[args.method](args).to(device)

In [24]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = CIFAR100(args.root, transform=preprocess, download=True, train=True)

val_dataset = CIFAR100(args.root, transform=preprocess, download=True, train=False)

train_loader = DataLoader(train_dataset,
                          batch_size=args.batch_size, pin_memory=True,
                          num_workers=args.num_workers, shuffle=True)

val_loader = DataLoader(val_dataset,
                        batch_size=args.batch_size, pin_memory=True,
                        num_workers=args.num_workers, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [27]:
# create data
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = CIFAR100(args.root, transform=preprocess,
                         download=True, train=True)

val_dataset = CIFAR100(args.root, transform=preprocess,
                       download=True, train=False)

train_loader = DataLoader(train_dataset,
                          batch_size=args.batch_size, pin_memory=True,
                          num_workers=args.num_workers, shuffle=True)

val_loader = DataLoader(val_dataset,
                        batch_size=args.batch_size, pin_memory=True,
                        num_workers=args.num_workers, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified
